In [15]:
import os
from os.path import join as pjoin

from bs4 import BeautifulSoup, SoupStrainer

In [16]:
path_brutos = pjoin(os.getcwd(), 'artigos_brutos')

In [17]:
soma = sum(1 for anos in os.listdir(path_brutos) for _ in os.listdir(pjoin(path_brutos, anos)))
print(f'Quantidade de artigos brutos: {soma}')

Quantidade de artigos brutos: 1681


# Teste com artigos de 2015 até 2022
Esses artigos são mais simples de limpar

In [18]:
soma = sum(1 for anos in range(2015,2023) for _ in os.listdir(pjoin(path_brutos, str(anos))))
print(f'Quantidade de artigos de 2015 até 2022: {soma}')

Quantidade de artigos de 2015 até 2022: 810


In [33]:
def limpa_html(text_html) -> str:
    soup = BeautifulSoup(text_html, 'lxml', parse_only=SoupStrainer('div', attrs={'data-anchor': 'Text'}))

    # tags do título, citações, referências, equações e figuras foram removidas.
    remover = soup.find_all('h1') + soup.find_all('span', attrs={'class': 'ref'}) + \
              soup.find_all('div', attrs={'class':'row formula'}) + soup.find_all('div', attrs={'class': 'row fig'}) + \
              soup.find_all('math')

    for tag in remover:
        tag.decompose()

    return ' '.join(soup.text.split())

In [35]:
documentos = []
for ano in range(2015, 2023):
    key = str(ano)

    for arquivo in os.listdir(pjoin(path_brutos, key)):
        path_leitura = pjoin(path_brutos, key, arquivo)

        with open(path_leitura, 'r', encoding='utf8') as f:
            texto_html = f.read()

        documentos.append(limpa_html(texto_html))

In [ ]:
import spacy
nlp = spacy.load('pt_core_news_lg')

docs_token = []
for doc in documentos:
    text = nlp(doc)
    docs_token.append([token.lemma_.lower() for token in text if not token.is_punct and not token.like_num and not token.text.lower() in nlp.Defaults.stop_words])


In [108]:
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 5 times or more).
bigram = Phrases(docs_token, min_count=5)
for idx in range(len(docs_token)):
    for token in bigram[docs_token[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs_token[idx].append(token)

In [114]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs_token)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.4)

In [115]:
corpus = [dictionary.doc2bow(doc) for doc in docs_token]

In [116]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 5524
Number of documents: 810


In [117]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 500
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)